In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os


In [85]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [86]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = os.path.join('/content/gdrive/MyDrive/Hackathon')
GOOGLE_DRIVE_PATH = os.path.join('gdrive', 'MyDrive',GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print('List files: ', os.listdir(GOOGLE_DRIVE_PATH))

List files:  ['clinical_dataset.csv', '.ipynb_checkpoints', 'clinical_dataset_with_days.csv', 'df5_cleaned.csv']


In [87]:
dataset_folder_path = GOOGLE_DRIVE_PATH

csv_files = [file for file in os.listdir(GOOGLE_DRIVE_PATH) if file.endswith('.csv')]


In [88]:
dataset = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH,'clinical_dataset.csv'))

In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'nct_id', 'outcome_type', 'measure', 'time_frame',
       'population', 'description'],
      dtype='object')

In [89]:
dataset.head(5)

,Unnamed: 0,id,nct_id,outcome_type,measure,time_frame,population,description
0,0,272006148,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,NaN,A patient-reported outcome measurement system ...
1,1,273155865,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,NaN,"Laboratory parameters include: Hematology, Cli..."
2,2,272450475,NCT01751113,primary,Area Under the Curve Calculated From 0 to 4 Ho...,Day 28 of each treatment period (up to 35 days),NaN,sGaw is a measure of airways conductance and i...
3,3,272682367,NCT05834790,primary,Baseline oral healthcare-related knowledge,This outcome will be measured before intervent...,NaN,Fifteen statements will be used to measure par...
4,4,272354029,NCT01813474,secondary,Tmax Following Single Dosing,"Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...",NaN,NaN


## Data Preprocessing- Shubham

In [ ]:
dataset.shape

(10000, 8)

In [ ]:
#Checking the null values
dataset.isnull().sum()

Unnamed: 0          0
id                  0
nct_id              0
outcome_type        0
measure             0
time_frame        168
population      10000
description      1946
dtype: int64

###Dropping Unwanted Columns

In [ ]:
dataset2 = dataset.drop(['id','population'],axis='columns')
dataset2.shape

(10000, 6)

In [ ]:
dataset3 = dataset2.iloc[:, 1:]
dataset3.head()


,nct_id,outcome_type,measure,time_frame,description
0,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,A patient-reported outcome measurement system ...
1,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,"Laboratory parameters include: Hematology, Cli..."
2,NCT01751113,primary,Area Under the Curve Calculated From 0 to 4 Ho...,Day 28 of each treatment period (up to 35 days),sGaw is a measure of airways conductance and i...
3,NCT05834790,primary,Baseline oral healthcare-related knowledge,This outcome will be measured before intervent...,Fifteen statements will be used to measure par...
4,NCT01813474,secondary,Tmax Following Single Dosing,"Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...",NaN


In [ ]:

df4 = dataset3.copy()
df4.head()

df4.isnull().sum()


nct_id             0
outcome_type       0
measure            0
time_frame       168
description     1946
dtype: int64

In [ ]:
df4_cleaned = df4.dropna(subset=['time_frame'])

df4_cleaned.isnull().sum()

nct_id             0
outcome_type       0
measure            0
time_frame         0
description     1779
dtype: int64

In [ ]:
df4_cleaned.shape

(9832, 5)

In [68]:
def convert_to_days(time_frame):
    if pd.isna(time_frame):
        return 'other'

    time_frame = time_frame.lower()

    # Check for specific keywords and handle them accordingly
    if 'single day' in time_frame or 'day' in time_frame:
        if 'days' in time_frame:
            parts = time_frame.split()
            for part in parts:
                if part.isdigit():
                    return int(part)
        elif 'day' in time_frame:
            return 1
    elif 'week' in time_frame:
        parts = time_frame.split()
        for part in parts:
            if part.isdigit():
                return int(part) * 7
    elif 'month' in time_frame:
        parts = time_frame.split()
        for part in parts:
            if part.isdigit():
                return int(part) * 30
    elif 'year' in time_frame:
        parts = time_frame.split()
        for part in parts:
            if part.isdigit():
                return int(part) * 365
    elif 'hour' in time_frame:
        return 1
    else:
        return 'other'



# Applying the conversion function to the "time_frame" column
df4_cleaned['time_frame_in_days'] = df4_cleaned['time_frame'].apply(convert_to_days)

# Displaying the DataFrame with selected columns
print(df4_cleaned[['time_frame', 'time_frame_in_days']].head(20))

df4_cleaned.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "clinical_dataset_with_days.csv"), index=False)



                                           time_frame time_frame_in_days
0      Pre-test at baseline and post-test at 10 weeks                 70
1                                          Single day                  1
2     Day 28 of each treatment period (up to 35 days)                 28
3   This outcome will be measured before intervent...              other
4   Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...                  1
5                                              1 year                365
6     After Engerix-B Kinder challenge dose (Month 1)               None
7                                            150 days                150
8                                            2 months                 60
9                                  24, 48 and72 hours                  1
10                                          12 months                360
11                                          12 Months                360
12                                            Month

<ipython-input-68-241a228334eb>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_cleaned['time_frame_in_days'] = df4_cleaned['time_frame'].apply(convert_to_days)


In [69]:
df4_cleaned.head(10)

,nct_id,outcome_type,measure,time_frame,description,time_frame_in_days
0,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,A patient-reported outcome measurement system ...,70
1,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,"Laboratory parameters include: Hematology, Cli...",1
2,NCT01751113,primary,Area Under the Curve Calculated From 0 to 4 Ho...,Day 28 of each treatment period (up to 35 days),sGaw is a measure of airways conductance and i...,28
3,NCT05834790,primary,Baseline oral healthcare-related knowledge,This outcome will be measured before intervent...,Fifteen statements will be used to measure par...,other
4,NCT01813474,secondary,Tmax Following Single Dosing,"Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...",NaN,1
5,NCT03979781,secondary,Patient-centered Outcomes - Satisfaction,1 year,consumer assessment of healthcare providers an...,365
6,NCT01333813,secondary,Number of Subjects Demonstrating an Anamnestic...,After Engerix-B Kinder challenge dose (Month 1),The anamnestic response is defined as an antib...,None
7,NCT04805333,primary,Recommended Phase II Dose,150 days,This study will determine the recommended phas...,150
8,NCT05137938,primary,Change in symptom severity of depression as me...,2 months,Hamilton Rating Scale for Depression (24-item ...,60
9,NCT01213680,primary,Total serum iron pharmakokinetic parameters,"24, 48 and72 hours",NaN,1


(8859, 6)

In [76]:
df5_cleaned = df4_cleaned.dropna(subset=['time_frame_in_days'])

In [81]:
df5_cleaned.shape

df5_cleaned.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "df5_cleaned.csv"), index=False)


In [80]:
df5_cleaned.head()

,nct_id,outcome_type,measure,time_frame,description,time_frame_in_days
0,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,A patient-reported outcome measurement system ...,70
1,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,"Laboratory parameters include: Hematology, Cli...",1
2,NCT01751113,primary,Area Under the Curve Calculated From 0 to 4 Ho...,Day 28 of each treatment period (up to 35 days),sGaw is a measure of airways conductance and i...,28
3,NCT05834790,primary,Baseline oral healthcare-related knowledge,This outcome will be measured before intervent...,Fifteen statements will be used to measure par...,other
4,NCT01813474,secondary,Tmax Following Single Dosing,"Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...",NaN,1


In [83]:
df5_cleaned.isnull().sum()

nct_id                   0
outcome_type             0
measure                  0
time_frame               0
description           1641
time_frame_in_days       0
dtype: int64